<a href="https://colab.research.google.com/github/bf319/Scaling_MPNNs/blob/main/Bogdan-Jonathan-Meeting-1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install -q git+https://github.com/deepmind/dm-haiku
%pip install -q jraph
%pip install -q git+https://github.com/deepmind/jaxline
%pip install -q ogb
%pip install -q dgl
%pip install -q optax
%pip install -q metis

     |████████████████████████████████| 75 kB 3.2 MB/s 
     |████████████████████████████████| 70 kB 3.9 MB/s 
     |████████████████████████████████| 77 kB 6.7 MB/s 
     |████████████████████████████████| 78 kB 3.7 MB/s 
     |████████████████████████████████| 4.4 MB 5.4 MB/s 
     |████████████████████████████████| 136 kB 5.3 MB/s 


In [ ]:
from ogb.nodeproppred import Evaluator
from ogb.nodeproppred import DglNodePropPredDataset

dataset = DglNodePropPredDataset(name = "ogbn-proteins")
split_idx = dataset.get_idx_split()
evaluator = Evaluator(name = 'ogbn-proteins')

DGL backend not selected or invalid.  Assuming PyTorch for now.
Using backend: pytorch


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


Downloaded 0.21 GB: 100%|██████████| 216/216 [00:07<00:00, 27.65it/s]


Extracting dataset/proteins.zip
Loading necessary files...
This might take a while.
Processing graphs...


100%|██████████| 1/1 [00:02<00:00,  2.29s/it]


Converting graphs into DGL objects...


100%|██████████| 1/1 [00:00<00:00,  2.31it/s]


Saving...


In [ ]:
# There is only one graph in Node Property Prediction datasets
ogbn_proteins_main_graph, ogbn_proteins_main_labels = dataset[0]

'''
  OGBN-Proteins
    #Nodes = 132,534
    #Edges = 39,561,252
    #Tasks = 112
    #Split Type = Species
    #Task Type = Binary classification
    #Metric = ROC-AUC

    Task:
      The task is to predict the presence of protein functions in a multi-label binary classification setup,
      where there are 112 kinds of labels to predict in total. 
      The performance is measured by the average of ROC-AUC scores across the 112 tasks.

    #Others:
      **undirected**
      **weighted**
      **typed (according to species)**

  (1) Nodes represent proteins
    (1.1) The proteins come from 8 species
      len(set(graph.ndata['species'].reshape(-1).tolist())) == 8
    (1.2) Each node has one feature associated with it (its species)
      graph.ndata['species'].shape == (#nodes, 1)
  
  (2) Edges indicate different types of biologically meaningful associations between proteins
    (2.1) All edges come with 8-dimensional features
      graph.edata['feat'].shape == (2 * #edges, 8)

'''
# Get split labels
train_label = dataset.labels[split_idx['train']]  # (86619, 112) -- binary values (presence of protein functions)
valid_label = dataset.labels[split_idx['valid']]  # (21236, 112) -- binary values (presence of protein functions)
test_label = dataset.labels[split_idx['test']]    # (24679, 112) -- binary values (presence of protein functions)

In [ ]:
import dgl

'''
  Generate graph partition using metis, with balanced number of edges in each partition.
  Note: 
    The subgraphs do not contain the node/edge data in the input graph (https://docs.dgl.ai/generated/dgl.metis_partition.html)
'''
num_partitions = 100
dgl_graph_metis_partition = dgl.metis_partition(ogbn_proteins_main_graph, num_partitions, balance_edges = True)

Convert a graph into a bidirected graph: 1.633 seconds
Construct multi-constraint weights: 0.436 seconds
Metis partitioning: 26.629 seconds
Split the graph: 0.444 seconds
Construct subgraphs: 0.041 seconds


In [ ]:
import numpy as np
import torch
import jraph
import jax.numpy as jnp

def dgl_graph_to_jraph(node_ids):
  # First add back the node and edge features
  dgl_graph_with_features = dgl.node_subgraph(ogbn_proteins_main_graph, node_ids)

  node_features = jnp.array(dgl_graph_with_features.ndata['species'])
  
  # I think it should be -- DGLGraph.edges uses the default (uv) format for edges
  # with u == source and v == destination
  # From https://docs.dgl.ai/generated/dgl.DGLGraph.edges.html#dgl.DGLGraph.edges
  senders = jnp.array(dgl_graph_with_features.edges()[0])
  receivers = jnp.array(dgl_graph_with_features.edges()[1])

  # Edges -- here we should include the 8-dimensional edge features
  edges = jnp.array(dgl_graph_with_features.edata['feat'])

  n_node = jnp.array([dgl_graph_with_features.num_nodes()])
  n_edge = jnp.array([dgl_graph_with_features.num_edges()])

  return jraph.GraphsTuple(
            nodes = node_features.astype(np.float32),
            senders = senders.astype(np.int32), 
            receivers = receivers.astype(np.int32),
            edges = edges.astype(np.float32),  
            n_node = n_node, 
            n_edge = n_edge,
            globals = None  # No global features
          )
  
def get_labels_for_subgraph(node_ids):
  return ogbn_proteins_main_labels.index_select(0, node_ids)

In [ ]:
import haiku as hk
import jax
import optax

from typing import Sequence

# See https://github.com/YuxuanXie/mcl/blob/5f7ee92e2a6bc89736263873a4ba9c14d1a676ff/glassy_dynamics/train_using_jax.py for alternative to using GraphMapFeatures
# From https://github.com/YuxuanXie/mcl/blob/5f7ee92e2a6bc89736263873a4ba9c14d1a676ff/glassy_dynamics/train_using_jax.py

mlp_sizes = (64, 128)
num_message_passing_steps = 3

@jraph.concatenated_args
def node_update_fn(feats: jnp.ndarray) -> jnp.ndarray:
  """Node update function for graph net."""
  net = hk.Sequential([hk.nets.MLP(output_sizes = mlp_sizes), jax.nn.relu, hk.LayerNorm(axis = -1, create_scale = False, create_offset = False)])
  return net(feats)

@jraph.concatenated_args
def edge_update_fn(feats: jnp.ndarray) -> jnp.ndarray:
  """Edge update function for graph net."""
  net = hk.Sequential([hk.nets.MLP(output_sizes = mlp_sizes), jax.nn.relu, hk.LayerNorm(axis = -1, create_scale = False, create_offset = False)])
  return net(feats)

def node_decoder_fn(feats: jnp.ndarray) -> jnp.ndarray:
  ## TODO -- Question: Should I use softmax here then round the decoded values? (true labels are either 0 or 1) 
  net = hk.Sequential([hk.nets.MLP(output_sizes = [112]), jax.nn.relu])
  return net(feats)

def network_definition(graph):
  """Defines a graph neural network.
  Args:
    graph: Graphstuple the network processes.
  Returns:
    Decoded nodes.
  """

  embedder = jraph.GraphMapFeatures(
      embed_node_fn=hk.Linear(mlp_sizes[0]),
      embed_edge_fn=hk.Linear(mlp_sizes[1]),
      )
  graph = embedder(graph)

  gn = jraph.InteractionNetwork(
      update_node_fn=node_update_fn,
      update_edge_fn=edge_update_fn,
      include_sent_messages_in_node_update=True
      )

  for _ in range(num_message_passing_steps):
    graph = gn(graph)

  decoder = jraph.GraphMapFeatures(embed_node_fn = node_decoder_fn)
  
  processed_graph = decoder(graph)
  return processed_graph.nodes

In [ ]:
import functools
import haiku as hk

from random import randint

# Try to follow this tutorial https://github.com/YuxuanXie/mcl/blob/5f7ee92e2a6bc89736263873a4ba9c14d1a676ff/glassy_dynamics/train_using_jax.py
def compute_loss(params, graph, label, net):
  decoded_nodes = net.apply(params, graph) # Shape == label.shape
  
  ## TODO -- Question: Should I round the predictions here? (predictions = jax.lax.round(decoded_nodes))
  predictions = decoded_nodes
  # (JG): optax has a binary cross entropy implementation.
  def compute_bce_with_logits_loss(x: jnp.ndarray, y: jnp.ndarray) -> jnp.ndarray:
    """
    Computes binary cross-entropy with logits loss.
    From https://colab.research.google.com/github/deepmind/educational/blob/master/colabs/summer_schools/intro_to_graph_nets_tutorial_with_jraph.ipynb#scrollTo=_Ld4b3D6Lwel

    Combines sigmoid and BCE, and uses log-sum-exp trick for numerical stability.
    See https://stackoverflow.com/a/66909858 if you want to learn more.

    Args:
      x: Predictions (logits).
      y: Labels.

    Returns:
      Binary cross-entropy loss with mean aggregation.

    """
    max_val = jnp.clip(x, 0, None)
    loss = x - x * y + max_val + jnp.log(jnp.exp(-max_val) + jnp.exp((-x - max_val)))
    return loss.mean()

  loss = compute_bce_with_logits_loss(predictions, label)
  return loss

def train(num_training_steps):
  # Transform the function (MPNN) into a pure function (with no side effects) so that it can be used with jax
  net = hk.without_apply_rng(hk.transform(network_definition))

  training_graph = dgl_graph_to_jraph(dgl_graph_metis_partition[0].ndata['_ID'])
  labels_training = get_labels_for_subgraph(dgl_graph_metis_partition[0].ndata['_ID'])
  labels_training = jnp.array(labels_training)

  params = net.init(jax.random.PRNGKey(42), training_graph)

  opt_init, opt_update = optax.adam(learning_rate = 1e-5)  # This learning rate could be a bit low, try 1e-1 for full graph descent?  
  opt_state = opt_init(params)

  # pmap?
  @jax.jit
  def update(params, opt_state, graph, targets):
    loss, grads = jax.value_and_grad(compute_loss)(params, graph, targets, net)
    updates, opt_state = opt_update(grads, opt_state)
    return optax.apply_updates(params, updates), opt_state, loss

  ## TODO - Question: How to train only using the train_split?


  # Train split, [0, 1, 10]
  # Train mask, jnp.zeros((num_nodes, 1)).at[train_split].set(1)
  # bce(pred, target) * train_mask

  # Preprocess data
  processed_data = []
  for idx in len(data):
    graph_idx = dgl_graph_to_jraph(dgl_graph_metis_partition[idx].ndata['_ID'])
    labels_idx = get_labels_for_subgraph(dgl_graph_metis_partition[idx].ndata['_ID'])
    labels_idx = jnp.array(labels_idx)


  # num training steps = 1000
  for idx in range(num_training_steps):
    graph = random.choice(processed_data)
    labels = random.choice(labels)
    params, opt_state, loss = update(params, opt_state, graph_idx, labels_idx)
    print('Loss training:', loss)
    # save parameters and opt state
  return params

def evaluate(params, num_graphs_eval):
  net = hk.without_apply_rng(hk.transform(network_definition))
  
  # Evaluate
  accumulated_loss = 0.0
  accumulated_roc = 0

  for idx in range(num_graphs_eval):
    graph_idx = dgl_graph_to_jraph(dgl_graph_metis_partition[idx].ndata['_ID'])
    labels_idx = get_labels_for_subgraph(dgl_graph_metis_partition[idx].ndata['_ID'])
    labels_idx = jnp.array(labels_idx)

    (loss, roc) = evaluate_graph(params, graph_idx, labels_idx, net)

    accumulated_loss += loss
    accumulated_roc += roc

  print(f'Average loss: {accumulated_loss / num_graphs_eval} | Average ROC: {accumulated_roc / num_graphs_eval}')

def evaluate_graph(params, graph, label, net):
  compute_loss_fn = jax.jit(functools.partial(compute_loss, net=net))

  decoded_nodes = net.apply(params, graph) # Shape == label.shape
  
  ## TODO -- Question: Should I round the predictions here? (predictions = jax.lax.round(decoded_nodes))
  predictions = decoded_nodes

  loss = compute_loss_fn(params, graph, label)
  roc = evaluator.eval({"y_true": np.array(label), "y_pred": np.array(predictions)})['rocauc']

  print(f'Eval loss: {loss} | ROC: {roc}')
  return (loss, roc)

final_params = train(num_training_steps = 10)
evaluate(final_params, 5)

Loss training: 0.86478597
Loss training: 0.86032945
Loss training: 0.8502318
Loss training: 0.829198
Loss training: 0.85182774
Loss training: 0.85189044
Loss training: 0.87464565
Loss training: 0.86070037
Loss training: 0.86909634
Loss training: 0.8350097
Eval loss: 0.8430590033531189 | ROC: 0.49673739182563603
Eval loss: 0.8407504558563232 | ROC: 0.5049564379095163
Eval loss: 0.8343844413757324 | ROC: 0.4970060154006341
Eval loss: 0.8171571493148804 | ROC: 0.49731982502274225
Eval loss: 0.8398522734642029 | ROC: 0.5001442550130821
Average loss: 0.8350406885147095 | Average ROC: 0.4992327850343221


In [ ]:
# First get your graph and convert it
# sharded_graphnet.graphs_tuple_to_broadcasted_sharded_graphs_tuple(
# Every tensor in the sharded graphs tuple has a first dimension of size [devices]
# Construct your sharded graph net, being clear to have num_shardes=num_devices
# sharded_gn = sharded_graphnet.ShardedEdgesGraphNetwork(
        # update_fn, update_fn, update_fn, num_shards=devices)
        
# @jax.pmap
# def psuedo_update(sharded_graph):
#   sharded_graph = sharded_gn(sharded_graph)
#   loss = bce(sharded_graph.nodes, targets) * target_mas
#   return loss


# Make sure to add jax.lax.pmean(grads) for your gradients!

# sharded_gn(sharded_graph)


In [ ]:
## TODO - Question: For data parallelism, my understanding is that I should apply vmap.
## Where should I apply it? In the computations of the losses? How would I then go about combining 
## the gradients before updating the parameters.

## TODO - Question: What to do about the error below with setting up pmap for Google Colab?
## I could use Kaggle which I understand gives access to TPUs v3 - v5
## but there is a long waiting time for running the code 
## and I would only have access for 20 hours of TPU time.

In [ ]:
# # Set up the TPU
# import jax
# import jax.tools.colab_tpu
# jax.tools.colab_tpu.setup_tpu()

# device_count = jax.device_count() # 8 devices

# jax.devices()

In [ ]:
# from jax.experimental import maps
# from jax.experimental import PartitionSpec
# from jax.experimental.pjit import pjit

# import numpy as np

# mesh_shape = (4, 2)
# devices = np.asarray(jax.devices()).reshape(*mesh_shape)

# mesh = maps.Mesh(devices, ('x', 'y'))
# mesh

In [ ]:

## TODO - Question
## Problem with pjit on Google Colabs
## https://github.com/google/jax/issues/8300

# from jax.experimental import maps
# from jax.experimental.pjit import PartitionSpec, pjit

# def test_pjit():
#   input_data = np.arange(8 * 2).reshape(8, 2)

#   f = pjit(
#     lambda x: x,
#     in_axis_resources=None,
#     out_axis_resources=PartitionSpec('x', 'y')
#     )
 
#   # Sends data to accelerators based on partition_spec
#   with maps.mesh(mesh.devices, mesh.axis_names):
#     data = f(input_data)

#     print(data)
#     print(data.device_buffers)

# test_pjit()

In [ ]:
# ## TODO: Take into account that nodes might have been reshufled
# def predict_labels(params, net):
#   y_pred = None

#   for i in range(num_partitions):
#     graph = dgl_graph_to_jraph(dgl_graph_metis_partition[i].ndata['_ID'])

#     if i == 0:
#       y_pred = np.array(jax.nn.softmax(net.apply(params, graph)))
#     else:
#       if i % 5 == 0:
#         print(f'Reached iteration {i}')
#       y_pred = np.append(
#           y_pred,
#           np.array(jax.nn.softmax(net.apply(params, graph))),
#           axis = 0
#       )

#   return y_pred

In [ ]:
# (_, true_labels) = dataset[0]
# print(true_labels.shape)
# print(torch.tensor(predicted_labels).shape)

In [ ]:
# from ogb.nodeproppred import Evaluator

# evaluator = Evaluator(name = 'ogbn-proteins')
# final_results = evaluator.eval({
#     'y_true': true_labels,
#     'y_pred': torch.tensor(predicted_labels)
# })
# print(final_results)
# # print(evaluator.expected_input_format) 
# # print(evaluator.expected_output_format) 